# Multilabel Classification

In [2]:
import matplotlib
import matplotlib.pyplot
import pathlib
import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection

matplotlib.pyplot.rc('font', size=14)
matplotlib.pyplot.rc('axes', labelsize=14, titlesize=14)
matplotlib.pyplot.rc('legend', fontsize=14)
matplotlib.pyplot.rc('xtick', labelsize=10)
matplotlib.pyplot.rc('ytick', labelsize=10)

output_dir = pathlib.Path() / "images" / "end_to_end_project"
output_dir.mkdir(parents=True, exist_ok=True)
print(f'output_dir : {output_dir}')

def matplotlib_to_imagefile(output_dir, filename, imgext="png", tight_layout=True, resolution=300):
    path = output_dir / f"{filename}.{imgext}"
    if tight_layout:
        matplotlib.pyplot.tight_layout()
    matplotlib.pyplot.savefig(path, format=imgext, dpi=resolution)

# 28 * 28 = 784
def plot_digit(vec_784):
    image = vec_784.reshape(28, 28)
    matplotlib.pyplot.imshow(image, cmap='binary')
    matplotlib.pyplot.axis('off')


dataset_save_dir = pathlib.Path("datasets")
mnist = sklearn.datasets.fetch_openml('mnist_784', data_home=dataset_save_dir, as_frame=False)
X, y = mnist.data, mnist.target
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.1, stratify = y)

output_dir : images\end_to_end_project


c:\Users\ctint\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
import numpy
import sklearn.neighbors

y_train_large = (y_train >= '7')
y_train_odd = (y_train.astype('int8') % 2 == 1)

# numpy.c_ : 예를 들어 numpy.c_[[1,2,3], [4,5,6]] 은 [1,4], [2,5], [3,6] 을 만든다.
y_multilabel = numpy.c_[y_train_large, y_train_odd]

knn_clf = sklearn.neighbors.KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

KNeighborsClassifier()

In [4]:
print(knn_clf.predict([X_train[0]]))

[[ True False]]


In [5]:
knn_clf = sklearn.neighbors.KNeighborsClassifier()
y_train_knn_pred = sklearn.model_selection.cross_val_predict(knn_clf, X_train, y_multilabel, cv=3, n_jobs=-1)

# 'macro': Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
print(sklearn.metrics.f1_score(y_multilabel, y_train_knn_pred, average="macro"))

0.977196048421094


In [6]:
# 'weighted': Calculate metrics for each label, and find their average weighted by support 
#             (the number of true instances for each label). This alters ‘macro’ to account 
#             for label imbalance; it can result in an F-score that is not between precision and recall.
# macro :    모든 label을 동등하게 생각해서 f1_score를 계산할 때에도 각 label에 대한 f1_score를 weight 없이 평균낸다
# weighted : 어떤 label에 True가 좀 더 많다면 그 label에 대한 weight를 좀 더 높게 잡는다. 즉 좀 더 데이터가 많은 label을 더 중시한다

print(sklearn.metrics.f1_score(y_multilabel, y_train_knn_pred, average="weighted"))

0.9785630304271272


In [7]:
import sklearn.multioutput
import sklearn.svm

# ClassifierChain을 사용하면 multilabel을 위해 만들어 지지 않은 classifier도 multilabel을 위해 사용할 수 있다
# 단순히 각각에 대해 train해서 독립적으로 사용하는게 아니라 순서를 정해서 앞선 classifier의 결과를 다음 classifier의 입력으로 사용한다
# 이렇게 하면 각 classifier가 앞선 classifier의 결과를 고려할 수 있게 된다
# 이런 방식은 각 classifier가 독립적으로 학습되는 것보다 성능이 좋을 수 있다
# 
# train시 앞선 classifier에 대한 결과를 true label에서 뽑아다 쓸것인가 predict한 결과를 가지고 쓸것인가를 선택할 수 있다
# default는 그냥 true label을 사용하는 것이다. cv를 지정하면 cross-validate된 predict한 결과를 사용 할 수 있다
chain_clf = sklearn.multioutput.ClassifierChain(sklearn.svm.SVC(), cv=3)
chain_clf.fit(X_train, y_multilabel) # 23m 30s

ClassifierChain(base_estimator=SVC(), cv=3)